# Libraries and Settings

In [4]:
import pandas as pd
import censusdis.data as ced

#pandas settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# ACS API call settings

In [70]:
acs5_profile = "acs/acs5/profile" # ACS 5-year profile estimates
year = 2021 # 2021 ACS 5-year estimates
all_states = "*" # all states
all_counties = "*"  # all counties

# Methods
1. Create variable dictonary to map variable names to variable descriptions
2. Use variable list to select variables from ACS table data
3. Include margin of error for each variable
4. Export as csv for merge with BRFSS data 


In [66]:
var_map = {
  #Population
  "DP05_0033E": "pop_tot", # Total population
  "DP02_0059E": "pop_25", # Estimate of total population 25 years and over (for educational attainment)
  "DP03_0026E": "pop_lf", # Estimate of (civilian) population 16 years and over in labor force (for occupation estimates)
  "DP03_0095E": "pop_hc", # Estimate of civilian noninstitutionalized population (for health insurance coverage estimates)

  #Total Households
  "DP02_0001E": "hh_tot", # Total households
  "DP02_0152E": "hh_tot_int", # Estimate of total households (for broadband internet subscription)
  "DP04_0057E": "hh_occu", # Estimate of occupied housing units (for transportation estimates)

  #Educational Attainment
  "DP02_0060PE": "pct_ed_lt9", # Estimate of educational attainment less than 9th grade
  "DP02_0061PE": "pct_ed_9_12", # Estimate of educational attainment 9th to 12th grade, no diploma
  "DP02_0062PE": "pct_ed_hs", # Estimate of educational attainment high school graduate (includes equivalency) 
  "DP02_0063PE": "pct_ed_sc", # Estimate of educational attainment some college, no degree
  "DP02_0064PE": "pct_ed_asc", # Estimate of educational attainment associate's degree
  "DP02_0065PE": "pct_ed_b", # Estimate of educational attainment bachelor's degree
  "DP02_0066PE": "pct_ed_gr", # Estimate of educational attainment graduate or professional degree

  "DP02_0060PM": "pct_m_ed_lt9", # Estimate of educational attainment less than 9th grade, margin of error
  "DP02_0061PM": "pct_m_ed_9_12", # Estimate of educational attainment 9th to 12th grade, no diploma, margin of error
  "DP02_0062PM": "pct_m_ed_hs", # Estimate of educational attainment high school graduate (includes equivalency) , margin of error
  "DP02_0063PM": "pct_m_ed_sc", # Estimate of educational attainment some college, no degree, margin of error
  "DP02_0064PM": "pct_m_ed_asc", # Estimate of educational attainment associate's degree, margin of error
  "DP02_0065PM": "pct_m_ed_b", # Estimate of educational attainment bachelor's degree, margin of error
  "DP02_0066PM": "pct_m_ed_gr", # Estimate of educational attainment graduate or professional degree, margin of error

  #Age 65 and over
  "DP05_0024PE": "pct_age_gte65", # Estimate of population age 65 and over
  
  "DP05_0024PM": "pct_m_age_gte65", # Estimate of population age 65 and over, margin of error

  #Race
  "DP05_0037PE" : "pct_race_white", #Estimate of total population white alone
  "DP05_0038PE" : "pct_race_black", #Estimate of total population black or African American alone
  "DP05_0039PE" : "pct_race_aian", #Estimate of total population American Indian and Alaska Native alone
  "DP05_0044PE" : "pct_race_asian", #Estimate of total population Asian alone
  "DP05_0052PE" : "pct_race_nhopi", #Estimate of total population Native Hawaiian and Other Pacific Islander alone
  "DP05_0057PE" : "pct_race_other", #Estimate of total population some other race

  "DP05_0037PM" : "pct_m_race_white", #Estimate of total population white alone, margin of error
  "DP05_0038PM" : "pct_m_race_black", #Estimate of total population black or African American alone, margin of error
  "DP05_0039PM" : "pct_m_race_aian", #Estimate of total population American Indian and Alaska Native alone, margin of error
  "DP05_0044PM" : "pct_m_race_asian", #Estimate of total population Asian alone, margin of error
  "DP05_0052PM" : "pct_m_race_nhopi", #Estimate of total population Native Hawaiian and Other Pacific Islander alone, margin of error
  "DP05_0057PM" : "pct_m_race_other", #Estimate of total population some other race, margin of error

  #Ethnicity
  "DP05_0071PE" : "pct_eth_hisp", #Estimate of total population Hispanic or Latino (any race)

  "DP05_0071PM" : "pct_m_eth_hisp", #Estimate of total population Hispanic or Latino, margin of error

  #Sex
  "DP05_0002PE" : "pct_sex_male", #Estimate of total population male
  "DP05_0003PE" : "pct_sex_female", #Estimate of total population female

  "DP05_0002PM" : "pct_m_sex_male", #Estimate of total population male, margin of error
  "DP05_0003PM" : "pct_m_sex_female", #Estimate of total population female, margin of error

  #Occupation
  "DP03_0005PE" : "pct_occ_unemp", # Estimate of civilian employed population 16 years unemployed
  "DP03_0027PE" : "pct_occ_mgt", # Estimate of civilian employed population 16 years and over in management, business, science, and arts occupations
  "DP03_0028PE" : "pct_occ_svc", # Estimate of civilian employed population 16 years and over in service occupations
  "DP03_0029PE" : "pct_occ_sales", # Estimate of civilian employed population 16 years and over in sales and office occupations
  "DP03_0030PE" : "pct_occ_nat_res", # Estimate of civilian employed population 16 years and over in natural resources, construction, and maintenance occupations
  "DP03_0031PE" : "pct_occ_prod", # Estimate of civilian employed population 16 years and over in production, transportation, and material moving occupations

  "DP03_0005PM" : "pct_m_occ_unemp", # Estimate of civilian employed population 16 years and over not in labor force, margin of error
  "DP03_0027PM" : "pct_m_occ_mgt", # Estimate of civilian employed population 16 years and over in management, business, science, and arts occupations, margin of error
  "DP03_0028PM" : "pct_m_occ_svc", # Estimate of civilian employed population 16 years and over in service occupations, margin of error
  "DP03_0029PM" : "pct_m_occ_sales", # Estimate of civilian employed population 16 years and over in sales and office occupations, margin of error
  "DP03_0030PM" : "pct_m_occ_nat_res", # Estimate of civilian employed population 16 years and over in natural resources, construction, and maintenance occupations, margin of error
  "DP03_0031PM" : "pct_m_occ_prod", # Estimate of civilian employed population 16 years and over in production, transportation, and material moving occupations, margin of error

  #Health Insurance Coverage
  "DP03_0099PE": "pct_hlth_unins", # Estimate of civilian noninstitutionalized population without health insurance coverage

  "DP03_0099PM": "pct_m_hlth_unins", # Estimate of civilian noninstitutionalized population without health insurance coverage, margin of error

  #Income (Estimates not available, percentage only)
  "DP03_0128PE": "pct_ses_pov", # Percentage estimate of all people whose income in past 12 is below poverty level; all people
  "DP03_0128PM": "pct_m_ses_pov", # Percentage estimate of all people whose income in past 12 is below poverty level; all people, margin of error
  #Transportation
  "DP04_0058PE": "pct_tp_veh_0", # Estimate of occupied housing units with no vehicles available
  "DP04_0059PE": "pct_tp_veh_1", # Estimate of occupied housing units with 1 vehicle available
  "DP04_0060PE": "pct_tp_veh_2", # Estimate of occupied housing units with 2 vehicles available
  "DP04_0061PE": "pct_tp_veh_3", # Estimate of occupied housing units with 3 vehicles available

  "DP04_0058PM": "pct_m_tp_veh_0", # Estimate of occupied housing units with no vehicles available, margin of error
  "DP04_0059PM": "pct_m_tp_veh_1", # Estimate of occupied housing units with 1 vehicle available, margin of error
  "DP04_0060PM": "pct_m_tp_veh_2", # Estimate of occupied housing units with 2 vehicles available, margin of error
  "DP04_0061PM": "pct_m_tp_veh_3", # Estimate of occupied housing units with 3 vehicles available, margin of error

  #Broadband Internet Subscription
  "DP02_0154PE": "pct_dg_bb_int", # Estimate of households with internet (Broadband) subscription

  "DP02_0154PM": "pct_m_dg_bb_int", # Estimate of households with internet (Broadband) subscription, margin of error
  
}
vars = ["NAME"] + list(var_map.keys())

df = ced.download(acs5_profile, year, vars, state = all_states, county = all_counties)
df = df.rename(columns=var_map)
df.columns = map(str.lower, df.columns)

In [67]:
df.head()

,state,county,name,pop_tot,pop_25,pop_lf,pop_hc,hh_tot,hh_tot_int,hh_occu,pct_ed_lt9,pct_ed_9_12,pct_ed_hs,pct_ed_sc,pct_ed_asc,pct_ed_b,pct_ed_gr,pct_m_ed_lt9,pct_m_ed_9_12,pct_m_ed_hs,pct_m_ed_sc,pct_m_ed_asc,pct_m_ed_b,pct_m_ed_gr,pct_age_gte65,pct_m_age_gte65,pct_race_white,pct_race_black,pct_race_aian,pct_race_asian,pct_race_nhopi,pct_race_other,pct_m_race_white,pct_m_race_black,pct_m_race_aian,pct_m_race_asian,pct_m_race_nhopi,pct_m_race_other,pct_eth_hisp,pct_m_eth_hisp,pct_sex_male,pct_sex_female,pct_m_sex_male,pct_m_sex_female,pct_occ_unemp,pct_occ_mgt,pct_occ_svc,pct_occ_sales,pct_occ_nat_res,pct_occ_prod,pct_m_occ_unemp,pct_m_occ_mgt,pct_m_occ_svc,pct_m_occ_sales,pct_m_occ_nat_res,pct_m_occ_prod,pct_hlth_unins,pct_m_hlth_unins,pct_ses_pov,pct_m_ses_pov,pct_tp_veh_0,pct_tp_veh_1,pct_tp_veh_2,pct_tp_veh_3,pct_m_tp_veh_0,pct_m_tp_veh_1,pct_m_tp_veh_2,pct_m_tp_veh_3,pct_dg_bb_int,pct_m_dg_bb_int
0,01,001,"Autauga County, Alabama",58239,39614.0,25871,56955,21856.0,21856.0,21856,2.0,8.4,32.8,19.6,9.1,16.4,11.7,0.6,1.1,2.2,2.0,1.4,1.8,1.4,15.1,0.2,75.1,19.7,0.2,1.1,0.0,0.3,0.5,0.5,0.1,0.4,0.1,0.3,3.0,NaN,48.4,51.6,0.4,0.4,1.6,38.1,15.1,21.9,9.5,15.5,0.5,2.9,2.0,2.2,1.5,1.6,7.9,1.2,13.6,2.0,5.4,31.5,33.8,29.3,1.3,2.6,2.3,2.5,85.5,1.7
1,01,003,"Baldwin County, Alabama",227131,161977.0,104367,224339,87190.0,87190.0,87190,2.1,6.9,27.4,21.7,9.5,20.6,11.8,0.3,0.8,1.2,1.1,0.8,1.1,0.9,20.6,0.1,84.5,8.8,0.6,1.0,0.0,1.8,0.6,0.3,0.2,0.2,0.1,0.6,4.7,NaN,48.8,51.2,0.1,0.1,2.2,36.1,17.8,23.6,9.5,13.0,0.3,1.5,1.4,1.5,1.1,1.2,9.7,1.0,9.2,0.9,2.4,31.7,43.2,22.7,0.4,1.6,1.6,1.5,87.9,1.0
2,01,005,"Barbour County, Alabama",25259,17995.0,8561,22295,9088.0,9088.0,9088,7.4,16.9,36.7,20.5,7.3,6.7,4.4,1.1,1.8,2.8,1.8,1.1,1.1,1.0,19.0,0.1,45.5,47.4,0.3,0.4,0.0,3.9,0.8,0.7,0.2,0.2,0.1,0.8,4.7,NaN,52.9,47.1,0.3,0.3,3.9,25.6,20.7,19.9,12.2,21.6,0.9,3.7,3.4,2.9,2.6,3.0,10.4,1.5,26.5,2.7,11.2,33.3,33.8,21.7,2.2,3.2,2.9,2.4,64.6,3.5
3,01,007,"Bibb County, Alabama",22412,16057.0,8223,21000,7083.0,7083.0,7083,6.2,13.3,43.9,18.0,6.7,7.9,4.0,1.7,2.4,3.3,2.6,1.3,2.4,1.3,16.0,0.5,75.9,22.3,0.1,0.2,0.0,0.1,0.5,0.6,0.2,0.3,0.2,0.1,2.8,NaN,54.9,45.1,0.7,0.7,4.8,22.0,16.8,21.5,16.6,23.1,1.3,3.8,4.4,4.9,3.8,4.6,10.0,2.0,16.9,3.6,6.3,24.7,37.7,31.3,3.1,4.1,4.3,4.6,74.6,4.1
4,01,009,"Blount County, Alabama",58884,40668.0,24244,58437,21300.0,21300.0,21300,6.8,9.6,35.1,21.5,12.1,9.3,5.6,0.8,1.2,1.8,1.5,1.3,1.2,0.8,18.0,0.2,92.5,1.3,0.3,0.4,0.1,2.1,1.0,0.3,0.2,0.1,0.1,0.8,9.5,NaN,50.1,49.9,0.2,0.2,3.3,28.2,13.8,21.6,15.3,21.1,0.8,2.2,1.6,2.3,1.7,2.2,10.0,1.3,13.2,1.7,4.5,25.0,32.1,38.4,1.2,2.4,2.3,2.6,81.0,1.9


## Export CSV including only percentage with CI

In [69]:
df.to_csv('acs5y_2021_pct_wCI.csv', index=False)

c:\ProgramData\miniconda3\envs\ds_mf\lib\site-packages\pandas\core\internals\blocks.py:2323: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)
